In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.4.0.post2 (c:\users\admin\anaconda3\lib\site-packages), Requirement.parse('azureml-core==1.0.83.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.4.0.post2 (c:\users\admin\anaconda3\lib\site-packages), Requirement.parse('azureml-core==1.0.83.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.4.0.post2 (c:\users\admin\anaconda3\lib\site-packages), Requirement.parse('azureml-core==1.0.83.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exception (azureml-

SDK version: 1.4.0


In [2]:
from azureml.core import Workspace
#ws = Workspace.create(name='ws01njgooglequest',
#                      subscription_id='1b7772b5-e1ea-49f0-8027-9fd1f6203aa1',
#                      resource_group='InnovationAIML-Nitin',
#                      create_resource_group=True,
#                      location='southeastasia'
#                     )
ws = Workspace.from_config('./.azureml/ws_config03.json')

In [5]:
#ws.write_config(path="./azure_config", file_name="ws_config.json")
#from azureml.core import Workspace
#ws_other_environment = Workspace.from_config(path="./file-path/ws_config.json")

In [3]:
import os
import os.path as path
project_root = path.abspath(path.join(os.getcwd()))
print(project_root)

C:\Users\admin\Documents\Nitin\mycodes\kaggle_google_quest_qna\lmtraining\distributed_v3


In [ ]:
from azureml.core import Datastore
ds = ws.get_default_datastore()

In [11]:
import os
#ds.upload(src_dir=project_root, target_path='njgooglequest03')
#ds.upload(src_dir=project_root+'\data\input', target_path='njgooglequest03/datasets')
#ds.upload(src_dir=project_root+'\input',  target_path='njgooglequest03/input')

#ds.upload(src_dir=project_root+'\data\input', target_path='njgooglequest05/datasets', show_progress=True)
#ds.upload(src_dir=project_root+'\model', target_path='njgooglequest05/model', show_progress=True)

#test
#ds.download(target_path='./test/',prefix='njgooglequest03/datasets/',show_progress=True)


Uploading an estimated of 0 files
Uploaded 0 files


$AZUREML_DATAREFERENCE_37581fd315014a289de4e17b8f30cc8a

In [5]:
from azureml.core import Experiment

experiment_name = 'njexp01_googlequest'
experiment = Experiment(ws, name=experiment_name)

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
gpu_cluster_name = "njgpucluster05"
gpu_compute_target = ComputeTarget(workspace=ws, name=gpu_cluster_name)

In [8]:
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import RunConfiguration
from azureml.core.container_registry import ContainerRegistry

run_user_managed = RunConfiguration()
run_user_managed.environment.python.user_managed_dependencies = True

# Define custom Docker image info
#image_name = 'mcr.microsoft.com/azureml/bert:pretrain-openmpi3.1.2-cuda10.0-cudnn7-ubuntu16.04'

estimator = PyTorch(source_directory='./code/source_dir_v2/',
                    compute_target=gpu_compute_target,
                     #Docker image
                    use_docker=True,
                    #custom_docker_image=image_name,
                    user_managed=False,
                    script_params = {
                          '--train_data_folder':ds.path('njgooglequest05/datasets/').as_mount(),
                          '--path_to_ckpt_config':ds.path('njgooglequest05/datasets/data/').as_mount(),
                          "--batch_size": 8,
                          '--num_epoch': 3,
                          '--lr': 1e-4,
                          '--lr_scheduler_name': "WarmupLinearSchedule",
                          '--seed': 44,
                          '--output_dir':ds.path('njgooglequest05/model/').as_mount(),
                    },
                    entry_script='./azure_train.py',
                    node_count=4,
                    #process_count_per_node=4,
                    distributed_training=azureml.train.dnn.Mpi(process_count_per_node=4),
                    use_gpu=True,
                    pip_requirements_file = './requirements2.txt'
                    #pip_packages=['transformers','scipy']
                   )

# path to the Python environment in the custom Docker image
#estimator._estimator_config.environment.python.interpreter_path = '/opt/miniconda/envs/amlbert/bin/python'


framework_version is not specified, defaulting to version 1.3.


In [9]:
run = experiment.submit(estimator)
from azureml.widgets import RunDetails
RunDetails(run).show()

A Jupyter Widget

In [10]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
njexp01_googlequest,njexp01_googlequest_1589430081_d85fb334,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation
